### По стандарту импортируем все необходимые модули и библиотки для решения задачи Seq2Seq для задачи машинного перевода:

In [2]:
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Постановка задачи:
----

Задача - научить RNNs переводить предложения с одного языка на другой язык. Корпус текстовых данных лежит в директории ./data/
Пока нам понадобиться только файл eng-fra.txt для того чтобы научить модель переводить с EN на FR язык.
Структура файла - обычный текстовой файл, состоящий из строк, где каждая строка 1 предложение на исходном языке (EN) Затем знак табуляции \t и предложение на целевом языке и так несколько строк в файле.
Нам необходимо сформировать искусственно класс словаря, провести кодировку последовательности и сформировать датасет.

## 1. Подготовим данные:

In [5]:
# Определим по умолчанию 2 токена которые будут нам информировать о начале предложения и конце предложения (SOS и EOS):
SOS_token = 0
EOS_token = 1

# Создадим объект словаря нашего языка, который будет хранить данные по маппингу слов - index2word и обратно word2index и плюс второстепенные методы по добавлению токена и обработке предложений:
class LanguageVocabulary(object):
    def __init__(self, name):
        # название языка
        self.name = name
        # словарик word2index который хранит соответственно кодировку слова в целочисленный индекс словаря
        self.word2index = {}
        # обычный словарик который хранит распределение слов, сколько слов мы использовали и сколько обнаружили
        self.word2count = {}
        # Обратный словарик словарю word2index где хранятся уже индексы и замаппенные слова к каждому индексу, нужен будет для расшифровки последовательности
        self.index2word = {0: "SOS", 1: "EOS"}
        # Count SOS and EOS, храним просто общее количество слов в нашем словаре, то есть количество токенов в сформированном словарике нашего языка
        self.n_words = 2

    def add_sentence(self, sentence):
        """
        Метод класса, для добавления предложения в словарь.
        Каждое предложение поступающее к нам, будет разбираться на
        примитивные токены и добавляться в словарь при помощи метода класса addword()
        """
        for word in sentence.split(' '):
            self.add_word(word)


    def add_word(self, word):
        # проверяем не входит ли наше слово в словарь word2index
        if word not in self.word2index:
            # добавляем в качестве ключа слово а в качестве значения последнее n_words
            self.word2index[word] = self.n_words
            # меняем на единичку
            self.word2count[word] = 1
            # и соответственно меняем и index2word словарик добавляя уже слово для декодирования
            self.index2word[self.n_words] = word
            # инкрементируем n_words
            self.n_words += 1
        else:
            # Если такое уже слово есть просто добавляем 1 что добавилось одно слово
            self.word2count[word] += 1

##### Класс словарика построили, теперь надо подумать как обработать последовательность текста. Создадим 2 фукнции uncode_to_ascii() и normalize_string(). Как мы помним из специфики работы Python, питоновский интерпретатор представляет строку в виде юникод кодировки, что неудобно. Необходимо перевести все в кодировку стандарта ASCII (FYI для разных языков, разный подход, прежде чем приступать к раскодировке из одного языка в другой, поищите информацию о представлении языка в памяти компьютера, например для Китайского, Японского, Иврита, Арабского и прочих языков есть своя определенная специфика). Обычно все насущные вопросы уже кем-то были решены и все решается просто поиском ответа на stackoverflow как я и сделал ниже:

In [7]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    # Декодируем из юникода в ascii
    s = unicode_to_ascii(s.lower().strip())
    # Что означает данное регулярное выражение - точку, !, ? меняем на пробел чтобы этот символ стоял отдельно от всех
    # https://docs.python.org/3/library/re.html - стандартная (родная) библиотка Python которая нужна для работы с регулярными выражениями
    s = re.sub(r"([.!?])", r" \1", s)
    # оставляем только наборы символов указанных в паттерне регулярного выражения остальное заменим на пробел
    s = re.sub(r"[^a-zA-Zа-яА-Я.!?]+", r" ", s)
    return s

##### Создадим функцию которая будет просто открывать наши документы корпуса и создавать словарик:

In [9]:
def read_languages(lang1, lang2, reverse=False):
    print("Reading lines...")
    # Берем документ корпуса, лежащий в директории ./data/___.txt подставляя значения указанных языков в нашем случае eng-fra, он читается бьется на предложения
    # lines = open('./data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    with open('./data/%s-%s.txt' % (lang1, lang2), mode='r', encoding='UTF8') as f:
        lines = f.read().split('\n')
        lines = lines[:-2]

    # Разбиваем построчно и нормализуем строку:
    pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]
    # Можем создавать и проходить как с целевого языка на исходный так и наоборот:
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = LanguageVocabulary(lang2)
        output_lang = LanguageVocabulary(lang1)
    else:
        input_lang = LanguageVocabulary(lang1)
        output_lang = LanguageVocabulary(lang2)
    return input_lang, output_lang, pairs

##### Чтобы немного упростить задачу и соответственно время на ее решение, для упрощения мы просто возьмем и отберем часть предложений которые будут начинаться на биграммы указанные в eng_prefixes и соответсвенно ограничим длину предложения документа 10-ю символами:

In [11]:
MAX_LENGTH = 10
# eng_prefixes = (
#     "i am ", "i m ",
#     "he is", "he s ",
#     "she is", "she s",
#     "you are", "you re ",
#     "we are", "we re ",
#     "they are", "they re "
# )

def filter_pair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

In [12]:
MAX_LENGTH = 20

def filter_pair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

#### Создадим функцию которая будет возвращать нам уже данные:

In [14]:
def prepare_data(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = read_languages(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filter_pairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [15]:
input_lang, output_lang, pairs = prepare_data('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 496059 sentence pairs
Trimmed to 495426 sentence pairs
Counting words...
Counted words:
rus 58492
eng 17158
['не стоит тебе пить .', 'you shouldn t be drinking .']


# 2. Построим  Encoder
-----------


In [17]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        # Как помним hidden_size - размер скрытого состояния
        self.hidden_size = hidden_size
        # Слой Эмбеддингов, который из входного вектора последовательности (либо батча) отдаст представление последовательности для скрытого состояния
        # FYI: в качестве Input_size у нас размер словаря
        self.embedding = nn.Embedding(input_size, hidden_size)
        # И соответственно рекуррентная ячейка GRU которая принимает MxM (hidden на hidden)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # Приводим эмбеддинг к формату одного предлоежния 1х1 и любая размерность
        embedded = self.embedding(input).view(1, 1, -1)
        # Нужно для следующего шага пока не запутываемся :) просто присвоили наш эмбеддинг
        output = embedded
        # и соответственно подаем все в GRU ячейку (эмбеддинг и скрытые состояния)
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        # Дополнительно сделаем инициализацию скрытого представления (просто заполним нулями)
        return torch.zeros(1, 1, self.hidden_size, device=device)

# 3. Построим  Decoder
-----------


In [19]:
class DecoderRNN(nn.Module):
    # Будьте внимательны, теперь на вход мы получаем размер скрытого представления
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # Слой эмбеддингов - рамер словаря, размер скрытого представления
        self.embedding = nn.Embedding(output_size, hidden_size)
        # GRU скрытое состояние на скрытое
        self.gru = nn.GRU(hidden_size, hidden_size)
        # Переводим hidden size в распределение для этого передаем в линейный слов скрытое состояни и размер словаря
        self.out = nn.Linear(hidden_size, output_size)
        # Получаем распределение верояностей
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0])) # берем output по нулевому индексу (одно предложение)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# 4. Создадим вспомогательные функции для работы с полученной репрезентацией и для кодирования для подачи в модель:

In [21]:
# Токены кодируем в целочисленное представление
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


# Берем предложение с указанным языком, делаем из него индексы и вставляем метку конца предложения, превращаем в тензор:
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# Для создания тензора из пар:
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# 5. Создадим функцию обучения для работы только с ОДНОЙ парой:

In [23]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    # Просто инициализируем скрытое представление для энкодера
    encoder_hidden = encoder.initHidden()
    # Скиыдваем градиенты для алгоритма градиентного спуска как и у энкодера так и у дэкодера
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Получаем размер в словаря (токенов) для входящего и выходящего тензора так как мы пробегаемся по каждому предложению по кусочкам
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    # Создаем переменную где будем хранить наши выходы из энкодера (в данной реализации пока не юзаем, далее будет еще один вариант)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    # пробегаем по длине входящего тензора и в экодер передаем последовательно каждый из токенов:
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        # Сохраняем все выходы из энкодера для одного слова
        encoder_outputs[ei] = encoder_output[0, 0]


    # Закончили с энкодером пошли к декодеру, как было сказано декодер начинается с SOS
    decoder_input = torch.tensor([[SOS_token]], device=device)
    # FYI здесь мы скрытое представление из энкодера передаем в скрытое представление в декодер, то есть после знака =
    # у нас будут ходить градиенты из декодера в энкодер, то есть когда мы будем считать градиенты, они сначала пробегут по декодеру
    # дойдут до знака = перескочат в энкодер и будут дальше считаться по энкодеру и эти градиенты сохранятся в соответствующих тензорах
    # и когда будут отрабатывать разные оптимайзеры (у нас их 2) у них будут соответствующие правильные градиенты которые смогут правильно отработать
    decoder_hidden = encoder_hidden

    # Будем использовать Teacher Forcing в части случае (подставляя правильную последовательность)
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        # Подаем decoder_input = torch.tensor([[SOS_token]], device=device) то есть по одному слову и скрытое представление
        for di in range(target_length):
            # Переведенное предложение и скрытое представление
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            # Считаем ошибку
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

# 6. Просто красивые функции для засекания времени:

In [25]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- eta: %s)' % (asMinutes(s), asMinutes(rs))

# 7. Функция которая будет пробегаться по всем парам и использовать эти пары для обучения сети:

In [27]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    # Делаем выборку наших пар функцией которую создали до
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    # FYI! Используем Negative Log-Likelihood Loss потому что log softmax уже присутствует в модели
    criterion = nn.NLLLoss()

    for epoch in range(1, n_iters + 1):
        training_pair = training_pairs[epoch - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        # Используем функцию для тренировки на отдельных токенах, которую написали выше
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_iters),
                                         epoch, epoch / n_iters * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    showPlot(plot_losses)

In [28]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.switch_backend('agg')


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# 8. Теперь построим функцию которая позволит нам использовать Encoder-Decoder для перевода предложения:

In [30]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for i in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
            encoder_outputs[i] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = [] # Наши деокдированные слова

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

In [31]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# 9. Этап обучения:

In [33]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
trainIters(encoder1, decoder1, 150_000, print_every=5000)

9m 17s (- eta: 269m 34s) (5000 3%) 4.4485
20m 6s (- eta: 281m 24s) (10000 6%) 3.9437
32m 20s (- eta: 291m 5s) (15000 10%) 3.6666
44m 44s (- eta: 290m 51s) (20000 13%) 3.4992
56m 12s (- eta: 281m 4s) (25000 16%) 3.3804
65m 40s (- eta: 262m 42s) (30000 20%) 3.1922
75m 10s (- eta: 246m 59s) (35000 23%) 3.1530
84m 12s (- eta: 231m 33s) (40000 26%) 3.0401
93m 53s (- eta: 219m 4s) (45000 30%) 3.0080
103m 29s (- eta: 206m 58s) (50000 33%) 2.9288
113m 3s (- eta: 195m 17s) (55000 36%) 2.8683
122m 41s (- eta: 184m 1s) (60000 40%) 2.8490
135m 18s (- eta: 176m 57s) (65000 43%) 2.7989
145m 33s (- eta: 166m 21s) (70000 46%) 2.7864
154m 47s (- eta: 154m 47s) (75000 50%) 2.7591
164m 5s (- eta: 143m 35s) (80000 53%) 2.7117
173m 6s (- eta: 132m 22s) (85000 56%) 2.7093
182m 5s (- eta: 121m 23s) (90000 60%) 2.6810
191m 7s (- eta: 110m 38s) (95000 63%) 2.6689
200m 6s (- eta: 100m 3s) (100000 66%) 2.6273
209m 6s (- eta: 89m 36s) (105000 70%) 2.6287
218m 2s (- eta: 79m 17s) (110000 73%) 2.6116
226m 59s (- et

# 10. А теперь все что мы сварганили протестим на работе:

In [35]:
evaluateRandomly(encoder1, decoder1)

> я ваш новыи сосед .
= i m your new neighbor .
< i m your new new . <EOS>

> у меня много фотографии .
= i have many photos .
< i have a lot of . <EOS>

> я просил вас не делать этого .
= i asked you not to do that .
< i didn not ask you to do that . <EOS>

> том не хочет быть хирургом .
= tom doesn t want to be a surgeon .
< tom doesn t want to be to . <EOS>

> ты так вырос !
= you ve grown so much !
< you re so so . <EOS>

> это самая большая глупость какую я когда либо слышал .
= that s the silliest thing i ve ever heard .
< this is my favorite time i ve ever heard . <EOS>

> я сеичас над этим работаю .
= i m working on it now .
< i m going now on work . <EOS>

> увидите .
= you will see .
< let s go . <EOS>

> том схватил руку мэри .
= tom grabbed mary s arm .
< tom made mary a <EOS>

> я решил ехать на машине .
= i decided to go by car .
< i m decided to go to bed . <EOS>



In [36]:
evaluateRandomly(encoder1, decoder1)

> ты предлагаешь другую теорию ?
= are you suggesting another theory ?
< do you have a to ? <EOS>

> я готов признать что это была моя вина .
= i m ready to admit it was my fault .
< i m the the was was on my fault . <EOS>

> хочешь изюма ?
= do you want some raisins ?
< do you want some milk ? <EOS>

> тому нужно подождать мэри .
= tom needs to wait for mary .
< tom needs to wait . <EOS>

> все о тебе беспокоятся .
= everybody s concerned about you .
< all of you to tell about the . <EOS>

> я хотел поговорить с матерью тома .
= i wanted to talk to tom s mother .
< i wanted to talk to tom with tom . <EOS>

> вообще то я не том .
= actually i m not tom .
< actually tom s not t t . . <EOS>

> дом сгорел дотла .
= the house burned down completely .
< the house is house in house . house . <EOS>

> я знаю где ваш дом .
= i know where your house is .
< i know where i house . <EOS>

> если бы была проблема том что нибудь сказал бы .
= if there had been a problem tom would ve said something .